In [101]:
import pandas as pd
import numpy as np
import requests
from googleAPIconfig import apiKey
import json
from datetime import datetime
from uszipcode import ZipcodeSearchEngine

In [102]:
search = ZipcodeSearchEngine()
#res = search.by_coordinate(39.122229, -77.133578, radius=1)

In [103]:
target_city = "Edison, NJ"

target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=33.9829, -118.3338&key=%s" % (apiKey)

target_url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (target_city, apiKey)
 
#latlng = "33.9829, -118.3338"
#latlngStr = latlng.strip("(").strip(")")
#target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=%s&key=%s" % (latlngStr,apiKey)
#geo_data = requests.get(target_url).json()

def createLatitude(latlng):
    try:
        #print(latlng)
        # This removes paranthesis 
        latlngArray = latlng.strip("(").strip(")").split(",")
        lat,lon = np.round(np.float(latlngArray[0]),3),np.round(np.float(latlngArray[1]),3)
        #print(lat,lon)
        return lat
    except:
        return ""
    
def createLongitude(latlng):
    try:
        #print(latlng)
        # This removes paranthesis 
        latlngArray = latlng.strip("(").strip(")").split(",")
        lat,lon = np.round(np.float(latlngArray[0]),3),np.round(np.float(latlngArray[1]),3)
        #print(lat,lon)
        return lon
    except:
        return ""

def getZipCode(latlng):
    #print(latlng)
    #latlngStr = latlng.strip("(").strip(")")
    #target_url1 = "https://maps.googleapis.com/maps/api/geocode/json?latlng=%s&key=%s" % (latlngStr,apiKey)
    #geo_data = requests.get(target_url1).json()
    #print(json.dumps(geo_data['results'][0], sort_keys=True, indent=4))
    try:
        #print([item['long_name'] for item in geo_data['results'][0]['address_components'] if "postal_code" in item['types']][0])
        #return [item['long_name'] for item in geo_data['results'][0]['address_components'] if "postal_code" in item['types']][0]
        latlngArray = latlng.strip("(").strip(")").split(",")
        lat,lon = np.float(latlngArray[0]),np.float(latlngArray[1])
        return search.by_coordinate(lat,lon, radius=1)[0]['Zipcode']
    except:
        #print(geo_data)
        return ""
    
def getDateTime(date_string):
    #print(date_string)
    date_list = [date_string.split("/")[2],date_string.split("/")[0],date_string.split("/")[1]]
    date_list = [np.int(item) for item in date_list]
    return datetime(date_list[0],date_list[1],date_list[2])

#Create a method that cleans the data
def checkResults(iteminSeries):
    if (type(iteminSeries)==str) & (iteminSeries != np.nan):
        return "True"
    else:
        #print(iteminSeries)
        return "False"

In [105]:
# Read csv file
df = pd.read_csv("Crime_Data_from_2010_to_Present.csv")

In [109]:
#Use the method to create a new column that return True/False strings to clean the data
df['Location Good'] = df['Location '].map(checkResults)

In [114]:
#Get the subset data in order to have zipCode entries for lat/lng coordinates
df2 = df[df['Location Good']=="True"]

In [115]:
#Create an column for the datetime of when crime occured
df2['Date Occurred (datetime)'] = df2["Date Occurred"].map(getDateTime)

/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [131]:
#Filter for the year 2016,2017,2018
df_2010 = df2[[item.year==2010 for item in df2["Date Occurred (datetime)"]]]
df_2011 = df2[[item.year==2011 for item in df2["Date Occurred (datetime)"]]]
df_2012 = df2[[item.year==2012 for item in df2["Date Occurred (datetime)"]]]

df_2013 = df2[[item.year==2013 for item in df2["Date Occurred (datetime)"]]]
df_2014 = df2[[item.year==2014 for item in df2["Date Occurred (datetime)"]]]
df_2015 = df2[[item.year==2015 for item in df2["Date Occurred (datetime)"]]]

df_2016 = df2[[item.year==2016 for item in df2["Date Occurred (datetime)"]]]
df_2017 = df2[[item.year==2017 for item in df2["Date Occurred (datetime)"]]]
df_2018 = df2[[item.year==2018 for item in df2["Date Occurred (datetime)"]]]

In [132]:
df_2010['Zip code'] = df_2010['Location '].map(getZipCode)
df_2010 = df_2010[df_2010["Zip code"]!=""]

df_2011['Zip code'] = df_2011['Location '].map(getZipCode)
df_2011 = df_2011[df_2011["Zip code"]!=""]

df_2012['Zip code'] = df_2012['Location '].map(getZipCode)
df_2012 = df_2012[df_2012["Zip code"]!=""]

df_2013['Zip code'] = df_2013['Location '].map(getZipCode)
df_2013 = df_2013[df_2013["Zip code"]!=""]

df_2014['Zip code'] = df_2014['Location '].map(getZipCode)
df_2014 = df_2014[df_2014["Zip code"]!=""]

df_2015['Zip code'] = df_2015['Location '].map(getZipCode)
df_2015 = df_2015[df_2015["Zip code"]!=""]

df_2016['Zip code'] = df_2016['Location '].map(getZipCode)
df_2016 = df_2016[df_2016["Zip code"]!=""]

df_2017['Zip code'] = df_2017['Location '].map(getZipCode)
df_2017 = df_2017[df_2017["Zip code"]!=""]

df_2018['Zip code'] = df_2018['Location '].map(getZipCode)
df_2018 = df_2018[df_2018["Zip code"]!=""]

/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/pgarias/Desktop/RutgersDataScienceBootcamp/python_environments/python3env/lib/python3.6/site-

In [135]:
print(len(df_2010),len(df_2011),len(df_2012),len(df_2013),len(df_2014),len(df_2015),len(df_2016))

150164 145063 146233 139994 143121 157849 164924


In [142]:
print(len(df_2010)-len(df_2010.drop_duplicates()))
print(len(df_2011)-len(df_2011.drop_duplicates()))
print(len(df_2012)-len(df_2012.drop_duplicates()))
print(len(df_2013)-len(df_2013.drop_duplicates()))
print(len(df_2014)-len(df_2014.drop_duplicates()))
print(len(df_2015)-len(df_2015.drop_duplicates()))
print(len(df_2016)-len(df_2016.drop_duplicates()))

0
0
0
0
0
0
0


In [147]:
df_2010.to_csv("2010_LA_Crimes.csv",index=False)
df_2011.to_csv("2011_LA_Crimes.csv",index=False)
df_2012.to_csv("2012_LA_Crimes.csv",index=False)
df_2013.to_csv("2013_LA_Crimes.csv",index=False)
df_2014.to_csv("2014_LA_Crimes.csv",index=False)
df_2015.to_csv("2015_LA_Crimes.csv",index=False)
df_2016.to_csv("2016_LA_Crimes.csv",index=False)


In [151]:
df_2016.groupby("Crime Code Description").count()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,MO Codes,Victim Age,...,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location,Location Good,Date Occurred (datetime),Zip code
Crime Code Description,,,,,,,,,,,,,,,,,,,,,
ARSON,310,310,310,310,310,310,310,310,228,204,...,310,50,0,0,310,27,310,310,310,310
ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER,137,137,137,137,137,137,137,137,136,73,...,137,36,0,0,137,77,137,137,137,137
"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",8430,8430,8430,8430,8430,8430,8430,8430,8368,8226,...,8430,1756,55,4,8430,3344,8430,8430,8430,8430
ATTEMPTED ROBBERY,960,960,960,960,960,960,960,960,940,894,...,960,126,6,0,960,477,960,960,960,960
BATTERY - SIMPLE ASSAULT,13844,13844,13844,13844,13844,13844,13844,13844,13774,13638,...,13844,301,4,0,13844,3143,13844,13844,13844,13844
BATTERY ON A FIREFIGHTER,32,32,32,32,32,32,32,32,31,27,...,32,4,0,0,32,17,32,32,32,32
BATTERY POLICE (SIMPLE),364,364,364,364,364,364,364,364,358,197,...,364,36,2,0,364,150,364,364,364,364
BATTERY WITH SEXUAL CONTACT,878,878,878,878,878,878,878,878,865,848,...,878,38,0,0,878,207,878,878,878,878
BIKE - ATTEMPTED STOLEN,8,8,8,8,8,8,8,8,8,7,...,8,2,0,0,8,3,8,8,8,8
